In [ ]:
!pip install -U torch transformers trl datasets umap-learn matplotlib seaborn pip  --q

In [7]:
!pip install git+https://github.com/Profluent-AI/E1.git --quiet


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.24.1 requires torch==2.9.1, but you have torch 2.8.0 which is incompatible.
torchaudio 2.9.0 requires torch==2.9.0, but you have torch 2.8.0 which is incompatible.


In [8]:
import pandas as pd

data = pd.read_csv("/users/nferruz/fstocco/Desktop/PETase-hackatlon/inputs/predictive-pet-zero-shot-test-2025.csv")
data.index.to_list()

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [4]:
import polars as pl

data = pl.read_csv("/users/nferruz/fstocco/Desktop/PETase-hackatlon/inputs")
data.head()

import E1.dist as dist
from E1.modeling import E1ForMaskedLM
from E1.predictor import E1Predictor

model_name = "Profluent-Bio/E1-300m"

model = E1ForMaskedLM.from_pretrained(model_name, dtype=torch.float).to(dist.get_device()).eval()
predictor = E1Predictor(
    model=model,
    max_batch_tokens=16384,  # Change to 4096 on Colab T4 GPU
    # We save token_embeddings of shape (Sequence Length, Embedding Dim) for all positions in the sequence.
    # and mean_token_embeddings of shape (Embedding Dim, ) which is the mean of token_embeddings over all positions.
    fields_to_save=["token_embeddings", "mean_token_embeddings"],
)

sequence_embeddings = []

for prediction in predictor.predict(
    sequences=data["sequence"].to_list(), sequence_ids=data.index.to_list(), context_seqs=None
):
    # Note, predictions may not be in the same order as the input sequences due to batching by length
    # Use prediction["id"] to match with the input sequences which is set to sequence id.
    org_name = prediction["id"]
    token_embeddings = prediction["token_embeddings"]  # (Sequence Length, Embedding Dim)
    mean_token_embeddings = prediction["mean_token_embeddings"]  # (Embedding Dim, )
    sequence_embeddings.append({"org_name": org_name, "sequence_embedding": mean_token_embeddings.cpu().numpy()})

sequence_embeddings = pl.DataFrame(sequence_embeddings).join(
    data.select(["org_name", "lid_type"]), on="org_name", how="inner"
)


CUDA 8.0 or higher detected; installing flash-attention


Scoring sequences against parent: 100%|██████████| 570/570 [00:00<00:00, 56420.29it/s]

[{'id': 0, 'context_id': 'mean', 'score': -3.345703125}, {'id': 1, 'context_id': 'mean', 'score': -11.28662109375}, {'id': 2, 'context_id': 'mean', 'score': -6.509765625}, {'id': 3, 'context_id': 'mean', 'score': -4.53125}, {'id': 4, 'context_id': 'mean', 'score': -18.5234375}, {'id': 5, 'context_id': 'mean', 'score': -9.65625}, {'id': 6, 'context_id': 'mean', 'score': -6.585937023162842}, {'id': 7, 'context_id': 'mean', 'score': -6.5}, {'id': 8, 'context_id': 'mean', 'score': -3.7724609375}, {'id': 9, 'context_id': 'mean', 'score': -11.41796875}, {'id': 10, 'context_id': 'mean', 'score': -4.911376953125}, {'id': 11, 'context_id': 'mean', 'score': -15.39453125}, {'id': 12, 'context_id': 'mean', 'score': -18.375}, {'id': 13, 'context_id': 'mean', 'score': -1.9084473848342896}, {'id': 14, 'context_id': 'mean', 'score': -4.54296875}, {'id': 15, 'context_id': 'mean', 'score': -18.28515625}, {'id': 16, 'context_id': 'mean', 'score': -6.7412109375}, {'id': 17, 'context_id': 'mean', 'score': 

In [5]:
import numpy as np
from umap import UMAP

sequence_embeddings_array = np.array(sequence_embeddings["sequence_embedding"].to_list())
lid_types = sequence_embeddings["lid_type"].to_list()

umap = UMAP(n_components=2, random_state=314)
sequence_embeddings_2d = umap.fit_transform(sequence_embeddings_array)

['...']

In [9]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.scatterplot(x=sequence_embeddings_2d[:, 0], y=sequence_embeddings_2d[:, 1], hue=lid_types)
plt.legend()
plt.show()